In [ ]:
import requests
import numpy as np
from pprint import pprint
from zipfile import ZipFile
import os
import json
from tqdm import tqdm
import shutil
from shutil import copyfile
!pip install opendatasets
import opendatasets as od
from pathlib import Path
from sklearn.model_selection import train_test_split
!pip install split-folders
import splitfolders

  Using cached opendatasets-0.1.22-py3-none-any.whl (15 kB)
  Using cached kaggle-1.5.13-py3-none-any.whl
  Using cached python_slugify-8.0.1-py2.py3-none-any.whl (9.7 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)

[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached split_folders-0.5.1-py3-none-any.whl (8.4 kB)

[notice] A new release of pip available: 22.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
def convert_bbox_coco2yolo(img_width, img_height, bbox):
    """
    Convert bounding box from COCO  format to YOLO format

    Parameters
    ----------
    img_width : int
        width of image
    img_height : int
        height of image
    bbox : list[int]
        bounding box annotation in COCO format:
        [top left x position, top left y position, width, height]

    Returns
    -------
    list[float]
        bounding box annotation in YOLO format:
        [x_center_rel, y_center_rel, width_rel, height_rel]
    """

    # YOLO bounding box format: [x_center, y_center, width, height]
    # (float values relative to width and height of image)
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

In [ ]:
def make_folders(path="output"):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path


def convert_coco_json_to_yolo_txt(output_path, json_file):

    path = make_folders(output_path)
    print(path)
    with open(json_file) as f:
        json_data = json.load(f)

    # write _darknet.labels, which holds names of all classes (one class per line)
    label_file = os.path.join(output_path, "_darknet.labels")
    with open(label_file, "w") as f:
        for category in tqdm(json_data["categories"], desc="Categories"):
            category_name = category["name"]
            f.write(f"{category_name}\n")

    for image in tqdm(json_data["images"], desc="Annotation txt for each iamge"):
        img_id = image["id"]
        img_name = image["file_name"]
        img_width = image["width"]
        img_height = image["height"]

        anno_in_image = [anno for anno in json_data["annotations"] if anno["image_id"] == img_id]
        anno_txt = os.path.join(output_path, img_name.split(".")[0] + ".txt")
        with open(anno_txt, "w") as f:
            for anno in anno_in_image:
                category = anno["category_id"]
                bbox_COCO = anno["bbox"]
                x, y, w, h = convert_bbox_coco2yolo(img_width, img_height, bbox_COCO)
                f.write(f"{category} {x:.6f} {y:.6f} {w:.6f} {h:.6f}\n")

    print("Converting COCO Json to YOLO txt finished!")

In [ ]:
def create_directory(s, l):
  folder = s
  directory = l
  path = os.path.join(directory, folder)
  os.mkdir(path)
  return

In [ ]:
#create datasets folder
dir_ = "/home/jovyan"
folder_ = "datasets"
create_directory(folder_,dir_)

In [ ]:
#download lincolnbeet dataset: 2 classes, sugar beet and weeds
%cd /home/jovyan/datasets
!wget https://www.dropbox.com/s/0rq7cc8t6rja632/all_fields_lincolnbeet.zip?dl=0
# specifying the zip file name
file_name = "all_fields_lincolnbeet.zip?dl=0"

# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

/home/jovyan/datasets
--2023-03-13 08:40:57--  https://www.dropbox.com/s/0rq7cc8t6rja632/all_fields_lincolnbeet.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/0rq7cc8t6rja632/all_fields_lincolnbeet.zip [following]
--2023-03-13 08:40:57--  https://www.dropbox.com/s/raw/0rq7cc8t6rja632/all_fields_lincolnbeet.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc76df01ab0a5ca0e916e84b86b1.dl.dropboxusercontent.com/cd/0/inline/B4KCf7PAxeRCjDAGrAS_8bltg_4SjXi66akoAqkGo_3EBgxb1QcFIlG0_a_QNEbSJCN-Y95yXaQ6OormeYQggVCGO8NBMPLicLoumRz6X1u4NmLspw87GjmNL_6RYXDKOQXTMA9Z0XRrJeUBGSdsQKsmttfh-RKItixR9qn8j8o1DA/file# [following]
--2023-03-13 08:40:57--  https://uc76df01ab0a5ca0e916e84b86b1.dl.dropboxusercontent.com/cd/0/inline/B4KCf7PA

In [ ]:
#create val, test and train folder
dir_test = "/home/jovyan/datasets/all_fields_lincolnbeet"
folder_test = "test"
create_directory(folder_test,dir_test)
dir_train = "/home/jovyan/datasets/all_fields_lincolnbeet"
folder_train = "train"
create_directory(folder_train,dir_train)
dir_val = "/home/jovyan/datasets/all_fields_lincolnbeet"
folder_val = "val"
create_directory(folder_val,dir_val)

In [ ]:
#create label folder with txt files instead of the json files for the Lincolnbeet dataset
%cd /home/jovyan/datasets/all_fields_lincolnbeet
yolo_val = "labels"
json_val = "json_val_set.json"
convert_coco_json_to_yolo_txt(yolo_val, json_val)
source = "/home/jovyan/datasets/all_fields_lincolnbeet/labels"
destination = "/home/jovyan/datasets/all_fields_lincolnbeet/val"
dest = shutil.move(source, destination)

In [ ]:
#create label folder with txt files instead of the json files for the Lincolnbeet dataset
%cd /home/jovyan/datasets/all_fields_lincolnbeet
yolo_test = "labels"
json_test = "json_test_set.json"
convert_coco_json_to_yolo_txt(yolo_test, json_test)
source = "/home/jovyan/datasets/all_fields_lincolnbeet/labels"
destination = "/home/jovyan/datasets/all_fields_lincolnbeet/test"
dest = shutil.move(source, destination)

In [ ]:
#create label folder with txt files instead of the json files for the Lincolnbeet dataset
%cd /home/jovyan/datasets/all_fields_lincolnbeet
yolo_train = "labels"
json_train = "json_train_set.json"
convert_coco_json_to_yolo_txt(yolo_train, json_train)
source = "/home/jovyan/datasets/all_fields_lincolnbeet/labels"
destination = "/home/jovyan/datasets/all_fields_lincolnbeet/train"
dest = shutil.move(source, destination)

In [ ]:
#copy images of the Lincolnbeet dataset to separate folder
source = "/home/jovyan/datasets/all_fields_lincolnbeet/all/"
dest = "/home/jovyan/datasets/all_fields_lincolnbeet"
dir_name = "images"
for file in os.listdir(source):
        if file.endswith(".png" or '.jpg'):
            try:
                # Create target Directory
                path = os.path.join(dest, dir_name)
                os.mkdir(path)
                # print("Directory " , dir_name ,  " Created ")
            except FileExistsError:
                # print("Directory " , dir_name ,  " already exists")
            shutil.copy((source+file) , path)

In [ ]:
#move image folder to train folder
source_dir = "/home/jovyan/datasets/all_fields_lincolnbeet/images"
destination_dir = "/home/jovyan/datasets/all_fields_lincolnbeet/train/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/all_fields_lincolnbeet/train/images'

In [ ]:
#move image folder to validation folder
source_dir = "/home/jovyan/datasets/all_fields_lincolnbeet/images"
destination_dir = "/home/jovyan/datasets/all_fields_lincolnbeet/val/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/all_fields_lincolnbeet/val/images'

In [ ]:
#create folder for chicory dataset
dir_chicory = "/home/jovyan/datasets"
folder_chicory = "chicory"
create_directory(folder_chicory,dir_chicory)

In [ ]:
#Get the Chicory dataset: one class named "weeds"
%cd /home/jovyan/datasets/chicory
!wget https://universe.roboflow.com/ds/BHnC2PDXti?key=EqMG6fZwud

In [ ]:
#download the Chicory dataset
%cd /home/jovyan/datasets/chicory
# specifying the zip file name
file_name = "BHnC2PDXti?key=EqMG6fZwud"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

In [ ]:
#move images from train folder to data_all in chicory dataset
source_dir = "/home/jovyan/datasets/chicory/train/images"
destination_dir = "/home/jovyan/datasets/chicory/data_all/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/chicory/data_all/images'

In [ ]:
#move labels from train folder to data_all in chicory dataset
source_dir = "/home/jovyan/datasets/chicory/train/labels"
destination_dir = "/home/jovyan/datasets/chicory/data_all/images"
shutil.copytree(source_dir, destination_dir)

In [ ]:
#manually added test folder data to data_all chicory dataset

In [ ]:
#split all labels from the Chicory dataset in training and validation sets with the ratio 70-30
import splitfolders
input_folder = r'/home/jovyan/datasets/chicory/data_all/labels_all/'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/chicory/data_splitted_new',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

Copying files: 3373 files [00:00, 10113.19 files/s]


In [ ]:
#move images from data_all in chicory dataset to data_splitted_new train data
source_dir = "/home/jovyan/datasets/chicory/data_all/images"
destination_dir = "/home/jovyan/datasets/chicory/data_splitted_new/train/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/chicory/data_splitted_new/train/images'

In [ ]:
#move images from data_all in chicory dataset to data_splitted_new val data
source_dir = "/home/jovyan/datasets/chicory/data_all/images"
destination_dir = "/home/jovyan/datasets/chicory/data_splitted_new/val/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/chicory/data_splitted_new/val/images'

In [ ]:
!pip install -q kaggle

In [ ]:
#WeedCrop dataset (Kaggle) --> folder "valid" has to be changed to "val"
%cd /home/jovyan/datasets
dir_weedcrop = "/home/jovyan/datasets"
folder_weedcrop = "WeedCrop"
create_directory(folder_weedcrop,dir_weedcrop)

/home/jovyan/datasets
/home/jovyan/datasets/WeedCrop
--2023-03-13 14:52:23--  https://www.kaggle.com/datasets/vinayakshanawad/weedcrop-image-dataset/download?datasetVersionNumber=1
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fvinayakshanawad%2Fweedcrop-image-dataset%2Fversions%2F1%3Fresource%3Ddownload [following]
--2023-03-13 14:52:23--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Fvinayakshanawad%2Fweedcrop-image-dataset%2Fversions%2F1%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘download?datasetVe

In [ ]:
#download the WeedCrop dataset
%cd /home/jovyan/datasets/WeedCrop
od.download("https://www.kaggle.com/datasets/vinayakshanawad/weedcrop-image-dataset/download?datasetVersionNumber=1")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  sophiewildeboer


Your Kaggle Key:

  ································


100%|██████████| 251M/251M [00:34<00:00, 7.58MB/s] 


In [ ]:
#move images of training to data_all folder for WeedCrop dataset
src_dir = "/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/train/images"
dest_dir = '/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/data_all/images'
files = os.listdir(src_dir)
shutil.copytree(src_dir, dest_dir)

'/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/tat_all/images'

In [ ]:
#move labels of training to data_all folder for WeedCrop dataset
src_dir = "/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/train/labels"
dest_dir = '/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/data_all/labels'
files = os.listdir(src_dir)
shutil.copytree(src_dir, dest_dir)

'/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/data_all/labels'

In [ ]:
#split the label files of the WeedCrop dataset in a training and validation set
import splitfolders
input_folder = r'/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/data_all/labels_1/'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/splitted/labels',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

Copying files: 2822 files [00:00, 5897.77 files/s]


In [ ]:
#move images from data_all to split train and val folder for WeedCrop dataset
src_dir = "/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/data_all/images_1/images"
dest_dir = '/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/splitted/images'
files = os.listdir(src_dir)
shutil.copytree(src_dir, dest_dir)

'/home/jovyan/datasets/WeedCrop/weedcrop-image-dataset/WeedCrop.v1i.yolov5pytorch/splitted/images'

In [ ]:
#Sesame dataset (Kaggle)
%cd /home/jovyan/datasets
dir_weedcrop = "/home/jovyan/datasets"
folder_weedcrop = "sesame"
create_directory(folder_weedcrop,dir_weedcrop)

/home/jovyan/datasets


In [ ]:
#download Sesame dataset
%cd /home/jovyan/datasets/sesam
od.download("https://www.kaggle.com/datasets/ravirajsinh45/crop-and-weed-detection-data-with-bounding-boxes/download?datasetVersionNumber=1")

/home/jovyan/datasets/sesam
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  sophiewildeboer


Your Kaggle Key:

  ································


100%|██████████| 79.2M/79.2M [00:13<00:00, 6.14MB/s]


In [ ]:
#Maize dataset (zenodo)
%cd /home/jovyan/datasets
dir_weedcrop = "/home/jovyan/datasets"
folder_weedcrop = "maize"
create_directory(folder_weedcrop,dir_weedcrop)

/home/jovyan/datasets


In [ ]:
#Download the Maize test dataset
%cd /home/jovyan/datasets/maize
od.download("https://zenodo.org/record/5106795/files/test_set.zip?download=1")

/home/jovyan/datasets/maize
--2023-03-14 14:51:08--  https://zenodo.org/record/5106795
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31764 (31K) [text/html]
Saving to: ‘5106795.3’

5106795.3           100%[===================>]  31.02K  --.-KB/s    in 0.001s  

2023-03-14 14:51:09 (45.6 MB/s) - ‘5106795.3’ saved [31764/31764]

Using downloaded and verified file: ./test_set.zip


In [ ]:
#Download the Maize train dataset
%cd /home/jovyan/datasets/maize
od.download("https://zenodo.org/record/5106795/files/train_set.zip?download=1")

/home/jovyan/datasets/maize


3352887296it [34:06, 1638279.54it/s]                                 


In [ ]:
#Download the Maize validation dataset
%cd /home/jovyan/datasets/maize
od.download("https://zenodo.org/record/5106795/files/validation_set.zip?download=1")

/home/jovyan/datasets/maize


1007771648it [11:43, 1432475.67it/s]                                


In [ ]:
#download the Maize test dataset
%cd /home/jovyan/datasets/maize
# specifying the zip file name
file_name = "test_set.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

/home/jovyan/datasets/maize
Extracting all the files now...
Done!


In [ ]:
#download the Maize test dataset
%cd /home/jovyan/datasets/maize
# specifying the zip file name
file_name = "train_set.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

/home/jovyan/datasets/maize
Extracting all the files now...
Done!


In [ ]:
#download the Maize test dataset
%cd /home/jovyan/datasets/maize
# specifying the zip file name
file_name = "validation_set.zip"
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

/home/jovyan/datasets/maize
Extracting all the files now...
Done!


In [ ]:
#copy labels of the Maize dataset to seperate folder
source_dir = "/home/jovyan/datasets/maize/train_set/labels"
destination_dir = "/home/jovyan/datasets/maize/all_labels"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/maize/all_labels'

In [ ]:
#copy all images of the Maize dataset to a separate folder
import shutil
from shutil import copyfile
source = "/home/jovyan/datasets/maize/original_data/test_set/"
dest = "/home/jovyan/datasets/maize"
dir_name = "images3"
for file in os.listdir(source):
        if file.endswith('JPG'):
            try:
                # Create target Directory
                path = os.path.join(dest, dir_name)
                os.mkdir(path)
                # print("Directory " , dir_name ,  " Created ")
            except FileExistsError:
                print("Directory " , dir_name ,  " already exists")
            shutil.copy((source+file) , path)

Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  already exists
Directory  images3  

In [ ]:
#combine classes 1 and 4 to crops and classes 0, 2, 3, 5, 6, 7, 8 and 9 to weed
%cd /home/jovyan/datasets/maize/all_labels
files = []
lst_weeds = ['0', '2', '3', '5', '6', '7', '8', '9']
# Add the path of txt folder
for i in os.listdir("/home/jovyan/datasets/maize/test_set/"):
    if i.endswith('.txt'):
        files.append(i)

for item in files:
    # define an empty list
    file_data = []

    # open file and read the content in a list
    with open(item, 'r') as myfile:
        for line in myfile:
            # remove linebreak which is the last character of the string
            currentLine = line[:-1]
            data = currentLine.split(" ")
            # add item to the list
            file_data.append(data)

    # Increase the first number in any line by one
    for i in file_data:
        if i[0].isdigit():
            print(i[0])
            if i[0] in lst_weeds:
                temp = 1
                i[0] = str(int(temp))
                # print(i[0])
            else:
                temp = 0
                i[0] = str(int(temp))
                # print(i[0])

    # Write back to the file
    f = open(item, 'w')
    for i in file_data:
        res = ""
        for j in i:
            res += j + " "
        f.write(res)
        f.write("\n")
    f.close()

In [ ]:
#split all labels of the Maize dataset in training and validation sets
import splitfolders
input_folder = r'/home/jovyan/datasets/maize/all_labels1/'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/maize',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

In [ ]:
#move image folder to validation folder of the Maize dataset; do this for the training set as well
source_dir = "/home/jovyan/datasets/maize/images"
destination_dir = "/home/jovyan/datasets/maize/val/images"
shutil.copytree(source_dir, destination_dir)

In [ ]:
#Sugar beet and weeds dataset
%cd /home/jovyan/datasets
dir_weedcrop = "/home/jovyan/datasets"
folder_weedcrop = "sugarbeet"
create_directory(folder_weedcrop,dir_weedcrop)

/home/jovyan/datasets


In [ ]:
#download the Sugar beet and weed dataset
%cd /home/jovyan/datasets/sugarbeet
od.download("https://www.kaggle.com/datasets/wangyongkun/sugarbeetsandweeds/download?datasetVersionNumber=1")

/home/jovyan/datasets/sugarbeet
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  sophiewildeboer


Your Kaggle Key:

  ································


100%|██████████| 63.7M/63.7M [00:08<00:00, 8.35MB/s]


In [ ]:
#split all labels of the Sugar beet and weeds dataset into training and validation sets
import splitfolders
input_folder = r'/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/data_all/labels_all/'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

Copying files: 120 files [00:00, 5852.65 files/s]


In [ ]:
#move image folder to train folder of sugarbeet dataset
source_dir = "/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/data_all/images"
destination_dir = "/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/train/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/train/images'

In [ ]:
#move image folder to validation folder of sugarbeet dataset
source_dir = "/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/data_all/images"
destination_dir = "/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/val/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/val/images'

In [ ]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/*.png'):
        image_list.append(filename)
    for filename in glob.glob(dir_path + '/*.JPG'):
        image_list.append(filename)

    return image_list

def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(dir_path, output_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    in_file = open(dir_path + '/' + basename_no_ext + '.xml' ,encoding='UTF8')
    out_file = open(output_path + basename_no_ext + '.txt', 'w' ,encoding='UTF8')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

cwd = getcwd()

In [ ]:
#convert labels of the Sugarbeet train dataset from xml to txt
%cd /home/jovyan/datasets
dirs = ['train']
classes = ['sugarbeet', 'weed']
for dir_path in dirs:
    full_dir_path = '/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/train' + '/' + dir_path
    output_path = '/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/train' +'/labels/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')
    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)

    list_file.close()

    print("Finished processing: " + dir_path)

/home/jovyan/datasets
Finished processing: train


In [ ]:
#convert labels of the Sugarbeet validation dataset from xml to txt
%cd /home/jovyan/datasets
dirs = ['val']
classes = ['sugarbeet', 'weed']
for dir_path in dirs:
    full_dir_path = '/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/val' + '/' + dir_path
    output_path = '/home/jovyan/datasets/sugarbeet/sugarbeetsandweeds/data/val' +'/labels/'
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    image_paths = getImagesInDir(full_dir_path)
    list_file = open(full_dir_path + '.txt', 'w')
    for image_path in image_paths:
        list_file.write(image_path + '\n')
        convert_annotation(full_dir_path, output_path, image_path)

    list_file.close()

    print("Finished processing: " + dir_path)

/home/jovyan/datasets
Finished processing: val


In [ ]:
source_dir = "/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels"
destination_dir = "/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels_extra"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels_extra'

In [ ]:
#change all weed classes to 0 in Weed-Detection-v2 dataset
%cd datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels
files = []
lst_weeds = ['0', '4', '5', '6', '7']
# Add the path of txt folder
for i in os.listdir("/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels/"):
    if i.endswith('.txt'):
        files.append(i)

for item in files:
    # define an empty list
    file_data = []

    # open file and read the content in a list
    with open(item, 'r') as myfile:
        for line in myfile:
            # remove linebreak which is the last character of the string
            currentLine = line[:-1]
            data = currentLine.split(" ")
            # add item to the list
            file_data.append(data)

    # Increase the first number in any line by one
    for i in file_data:
        if i[0].isdigit():
            # print(i[0])
            if i[0] in lst_weeds:
                temp = 0 #float(i[0]) + 1
                i[0] = str(int(temp))
                # print(i[0])
            else:
                temp = 1 #float(i[0]) + 1
                i[0] = str(int(temp))
                # print(i[0])
    # Write back to the file
    f = open(item, 'w')
    for i in file_data:
        res = ""
        for j in i:
            res += j + " "
        f.write(res)
        f.write("\n")
    f.close()

/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/Final Year - Weed detection.v1-finalyearannotations1.yolov5pytorch/train/labels


In [ ]:
#split all labels in training and validation sets
%cd /home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/FinalYear-Weeddetection.v1-finalyearannotations1.yolov5pytorch/all_data/labels_1
import splitfolders
input_folder = r'/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/FinalYear-Weeddetection.v1-finalyearannotations1.yolov5pytorch/all_data/labels_1'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/FinalYear-Weeddetection.v1-finalyearannotations1.yolov5pytorch/all_data',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

[Errno 2] No such file or directory: '/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/FinalYear-Weeddetection.v1-finalyearannotations1.yolov5pytorch/all_data/labels'
/home/jovyan/datasets/Weed_Detection_v2/weed-detection-v2/FinalYear-Weeddetection.v1-finalyearannotations1.yolov5pytorch/all_data/labels


Copying files: 217 files [00:00, 6122.78 files/s]


In [ ]:
#copy labels of sesame dataset to seperate folder
source = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/data/"
dest = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/"
dir_name = "labels"
for file in os.listdir(source):
        if file.endswith('.txt'):
            try:
                # Create target Directory
                path = os.path.join(dest, dir_name)
                os.mkdir(path)
                # print("Directory " , dir_name ,  " Created ")
            except FileExistsError:
                print("Directory " , dir_name ,  " already exists")
            shutil.copy((source+file) , path)

Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  labels  already exists
Directory  lab

In [ ]:
#split all labels of the Sesame dataset into training and validation sets
import splitfolders
input_folder = r'/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/labels'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

Copying files: 1300 files [00:00, 8389.46 files/s]


In [ ]:
#copy all images of sesame dataset to separate folder
source = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/data/"
dest = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data"
dir_name = "images"
for file in os.listdir(source):
        if file.endswith('jpeg'):
            try:
                # Create target Directory
                path = os.path.join(dest, dir_name)
                os.mkdir(path)
                # print("Directory " , dir_name ,  " Created ")
            except FileExistsError:
                print("Directory " , dir_name ,  " already exists")
            shutil.copy((source+file) , path)

Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory  image  already exists
Directory 

In [ ]:
#move image folder to train folder of sesame dataset
source_dir = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/images"
destination_dir = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/train/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/train/images'

In [ ]:
#move image folder to val folder of sesame dataset
source_dir = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/images"
destination_dir = "/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/val/images"
shutil.copytree(source_dir, destination_dir)

'/home/jovyan/datasets/WeedDetection (sesam)/weed-detection/archive-2/agri_data/val/images'

In [ ]:
#download the CottonWeedDet12 dataset
%cd /home/jovyan/datasets/CottonWeedDet12
od.download("https://zenodo.org/record/7535814/files/CottonWeedDet12.7z?download=1")

/home/jovyan/datasets/CottonWeedDet12


28586475520it [4:18:29, 1843124.92it/s]                                   


In [ ]:
#unzip CottonWeedDet12
!pip install py7zr
from py7zr import unpack_7zarchive
import shutil

shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
shutil.unpack_archive('CottonWeedDet12.7z', '/home/jovyan/datasets/CottonWeedDet12')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.4/390.4 kB 4.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
#change all classes to 0 for CottonWeedDet12 dataset
%cd /home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12/annotation_YOLO_txt
files = []
lst_weeds = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# Add the path of txt folder
for i in os.listdir("/home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12/annotation_YOLO_txt/"):
    if i.endswith('.txt'):
        files.append(i)

for item in files:
    # define an empty list
    file_data = []

    # open file and read the content in a list
    with open(item, 'r') as myfile:
        for line in myfile:
            # remove linebreak which is the last character of the string
            currentLine = line[:-1]
            data = currentLine.split(" ")
            # add item to the list
            file_data.append(data)

    # Increase the first number in any line by one
    for i in file_data:
        if i[0].isdigit():
            # print(i[0])
            if i[0] in lst_weeds:
                temp = 0 #float(i[0]) + 1
                i[0] = str(int(temp))
                # print(i[0])
            else:
                temp = 0 #float(i[0]) + 1
                i[0] = str(int(temp))
                # print(i[0])
    # Write back to the file
    f = open(item, 'w')
    for i in file_data:
        res = ""
        for j in i:
            res += j + " "
        f.write(res)
        f.write("\n")
    f.close()

/home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12/annotation_YOLO_txt


In [ ]:
#split all labels in training and validation sets
import splitfolders
input_folder = r'/home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12/labels/'
splitfolders.ratio(input_folder, output=r'/home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12',
                   seed=0, ratio=(.7, .3),
                   group_prefix=None) # default values

Copying files: 5648 files [00:00, 8331.71 files/s] 


In [ ]:
#merge folders; use this to merge all the datasets
# Function to create new folder if not exists
%cd /home/jovyan/datasets/chicory_sesam_sugarbeet
def make_new_folder(folder_name, parent_folder):

    # Path
    path = os.path.join(parent_folder, folder_name)

    # Create the folder
    # 'new_folder' in
    # parent_folder
    try:
        # mode of the folder
        mode = 0o777

        # Create folder
        os.mkdir(path, mode)
    except OSError as error:
        print(error)

# current folder path
current_folder = os.getcwd()

# list of folders to be merged
list_dir = ['chicory_train', 'sesam_train', 'sugarbeet_train']

# enumerate on list_dir to get the
# content of all the folders ans store
# it in a dictionary
content_list = {}
for index, val in enumerate(list_dir):
    path = os.path.join(current_folder, val)
    content_list[ list_dir[index] ] = os.listdir(path)

# folder in which all the content will
# be merged
merge_folder = "images"

# merge_folder path - current_folder
# + merge_folder
merge_folder_path = os.path.join(current_folder, merge_folder)

# create merge_folder if not exists
make_new_folder(merge_folder, current_folder)

# loop through the list of folders
for sub_dir in content_list:

    # loop through the contents of the
    # list of folders
    for contents in content_list[sub_dir]:

        # make the path of the content to move
        path_to_content = sub_dir + "/" + contents

        # make the path with the current folder
        dir_to_move = os.path.join(current_folder, path_to_content )

        # move the file
        shutil.move(dir_to_move, merge_folder_path)

/home/jovyan/datasets/chicory_sesam_sugarbeet
images
labels.cache
labels
images


Error: Destination path '/home/jovyan/datasets/chicory_sesam_sugarbeet/images/images' already exists

In [ ]:
#download grapes train dataset; do this for the test and validation set as well
import zipfile
%cd /home/jovyan/datasets/grapes_right
# specifying the zip file name
file_name = "train.zip"
# opening the zip file in READ mode
with zipfile.ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    # zip.printdir()
    # extracting all the files
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

/home/jovyan/datasets/grapes_right
Extracting all the files now...
Done!


In [ ]:
import os
total_size = 0
start_path = r"/home/jovyan/datasets/CottonWeedDet12/CottonWeedDet12/val"
#get size of current directory
for path, dirs, files in os.walk(start_path):
    for f in files:
        fp = os.path.join(path,f)
        total_size += os.path.getsize(fp)
print("Drictory size:" + str(total_size))
print('Drictory size: {:,} MB'.format(int(total_size/1024**2)).replace(',', ' '))
print('Drictory size: {:,} GB'.format(int(total_size/1024**3)).replace(',', ' '))

Drictory size:8683409589
Drictory size: 8 281 MB
Drictory size: 8 GB
